<a href="https://colab.research.google.com/github/MayerT1/ACI_Collaboration/blob/main/ACI_cerulean_warbler_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pygbif

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.2/70.2 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.0/70.0 kB 5.5 MB/s eta 0:00:00


In [2]:
import ee
import geemap
import pandas as pd
from pygbif import occurrences
import geemap.chart as chart

In [3]:
# Authenticate and initialize Earth Engine
ee.Authenticate()
ee.Initialize(project='servir-sco-assets')
Map = geemap.Map()

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_7TDKVSyKvBdmMqW?ref=4i2o6


https://www.gbif.org/species/6093350

In [5]:
# coords = [
#     [-85.85307177259287, 35.06788356661989],
#     [-85.41087206556162, 35.06788356661989],
#     [-85.41087206556162, 35.34391870157585],
#     [-85.85307177259287, 35.34391870157585],
#     [-85.85307177259287, 35.06788356661989]  # Close the polygon
# ]

# roi = ee.Geometry.Polygon(coords)

# # Create a map centered on the ROI
# Map = geemap.Map(center=[35.2, -85.63], zoom=11)
# Map.addLayer(roi, {"color": "red"}, "Modeling ROI")
# # Map

Map(center=[35.2, -85.63], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGU…

In [ ]:
# Define AOI: Tennessee
states = ee.FeatureCollection("TIGER/2018/States")
tennessee = states.filter(ee.Filter.eq('NAME', 'Tennessee'))
roi = tennessee.geometry()
Map = geemap.Map(center=[35, -86], zoom=6)
Map.addLayer(tennessee, {}, "Tennessee AOI")

# --------------------------------------------------
# Fetch Cerulean Warbler occurrences from GBIF
# --------------------------------------------------
species_name = "Setophaga cerulea"
gbif_data = occurrences.search(scientificName=species_name, country="US", stateProvince="Tennessee", hasCoordinate=True, limit=3000)

# Convert GBIF occurrences to a DataFrame
records = gbif_data['results']
df = pd.DataFrame(records)
df = df.dropna(subset=['decimalLatitude', 'decimalLongitude'])

# Convert to ee.FeatureCollection
def df_to_fc(row):
    return ee.Feature(ee.Geometry.Point([row['decimalLongitude'], row['decimalLatitude']]), {'occurrence': 1})

presence_fc = ee.FeatureCollection(df.apply(df_to_fc, axis=1).tolist())
Map.addLayer(presence_fc, {'color': 'blue'}, "Presence Points")

# --------------------------------------------------
# Generate background (pseudo-absence) points
# --------------------------------------------------
background_fc = ee.FeatureCollection.randomPoints(roi, 10000, seed=42)
background_fc = background_fc.map(lambda f: f.set('occurrence', 0))
Map.addLayer(background_fc, {'color': 'red'}, "Background Points")

# Merge presence and background points
training_data = presence_fc.merge(background_fc)
Map.addLayer(training_data, {}, "Training Data")

# --------------------------------------------------
# Landsat 8 imagery preprocessing
# --------------------------------------------------
def maskL8sr(image):
    qa = image.select('QA_PIXEL')
    cloudShadowBitMask = 1 << 3
    cloudsBitMask = 1 << 5
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(
        qa.bitwiseAnd(cloudsBitMask).eq(0)
    )
    return image.updateMask(mask)

LS8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2") \
        .filterDate('2020-01-01', '2020-12-31') \
        .filterBounds(roi) \
        .map(maskL8sr)

# Compute NDVI median image
def add_ndvi(image):
    ndvi = image.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')
    return ndvi.copyProperties(image).set('system:time_start', image.get('system:time_start'))

image = LS8.map(add_ndvi).median().clip(roi)
Map.addLayer(image, {'min': 0, 'max': 1, 'palette': ['white', 'green']}, "NDVI")

# --------------------------------------------------
# Sample points and train Maxent classifier
# --------------------------------------------------
training = image.sampleRegions(
    collection=training_data,
    scale=30
)

classifier = ee.Classifier.amnhMaxent().train(
    features=training,
    classProperty='occurrence',
    inputProperties=image.bandNames()
)

# Classify image
image_classified = image.classify(classifier)
# Map.addLayer(image_classified.select('probability'), {'min':0, 'max':1, 'palette': ['white','light green','green']}, 'Predicted Probability')

# Define a gradient from transparent to dark green
palette = ['#00ffffff', '#006400ff']  # transparent -> dark green

# Add layer
Map.addLayer(image_classified.select('probability'),
             {'min':0, 'max':1, 'palette': palette},
             'Predicted Probability')



# Display map
Map


In [9]:
# # --------------------------------------------------
# # Clip outputs for display only
# # --------------------------------------------------
# coords = [
#     [-85.85307177259287, 35.06788356661989],
#     [-85.41087206556162, 35.06788356661989],
#     [-85.41087206556162, 35.34391870157585],
#     [-85.85307177259287, 35.34391870157585],
#     [-85.85307177259287, 35.06788356661989]  # Close the polygon
# ]
# out_roi = ee.Geometry.Polygon(coords)

# # Clip layers for display
# presence_fc_disp = presence_fc.map(lambda f: f.intersection(out_roi, 1))
# background_fc_disp = background_fc.map(lambda f: f.intersection(out_roi, 1))
# training_data_disp = training_data.map(lambda f: f.intersection(out_roi, 1))
# image_disp = image.clip(out_roi)
# image_classified_disp = image_classified.clip(out_roi)

# # Add clipped layers to map
# Map.addLayer(image_disp, {'min':0, 'max':1, 'palette':['white','green']}, "NDVI (Clipped)")
# Map.addLayer(image_classified_disp.select('probability'), {'min':0, 'max':1, 'palette':['red','white','green']}, "Predicted Probability (Clipped)")
# Map.addLayer(presence_fc_disp, {'color':'blue'}, "Presence Points (Clipped)")
# Map.addLayer(background_fc_disp, {'color':'red'}, "Background Points (Clipped)")


In [ ]:
# import ee
# import geemap
# import pandas as pd
# from pygbif import occurrences

# ee.Initialize()

# -----------------------------
# Define AOI: Tennessee
# -----------------------------
states = ee.FeatureCollection("TIGER/2018/States")
tennessee = states.filter(ee.Filter.eq('NAME', 'Tennessee'))
roi = tennessee.geometry()
Map = geemap.Map(center=[35, -86], zoom=6)
Map.addLayer(tennessee, {}, "Tennessee AOI")

# # Define your custom ROI
# coords = [
#     [-85.85307177259287, 35.06788356661989],
#     [-85.41087206556162, 35.06788356661989],
#     [-85.41087206556162, 35.34391870157585],
#     [-85.85307177259287, 35.34391870157585],
#     [-85.85307177259287, 35.06788356661989]  # Close the polygon
# ]

# # Create EE Geometry and buffer by 10 km (10000 meters)
# roi = ee.Geometry.Polygon(coords).buffer(100000)


# -----------------------------
# GBIF occurrences for Cerulean Warbler
# -----------------------------
species_name = "Setophaga cerulea"
gbif_data = occurrences.search(scientificName=species_name, country="US", stateProvince="Tennessee", hasCoordinate=True, limit=3000)
records = gbif_data['results']
df = pd.DataFrame(records).dropna(subset=['decimalLatitude', 'decimalLongitude'])

def df_to_fc(row):
    return ee.Feature(ee.Geometry.Point([row['decimalLongitude'], row['decimalLatitude']]), {'occurrence': 1})

presence_fc = ee.FeatureCollection(df.apply(df_to_fc, axis=1).tolist())
Map.addLayer(presence_fc, {'color': 'blue'}, "Presence Points")

# -----------------------------
# Background points
# -----------------------------
background_fc = ee.FeatureCollection.randomPoints(roi, 10000, seed=42).map(lambda f: f.set('occurrence', 0))
training_data = presence_fc.merge(background_fc)
Map.addLayer(background_fc, {'color': 'red'}, "Background Points")

# -----------------------------
# Landsat 8 NDVI
# -----------------------------
def maskL8sr(image):
    qa = image.select('QA_PIXEL')
    cloudShadowBitMask = 1 << 3
    cloudsBitMask = 1 << 5
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(
        qa.bitwiseAnd(cloudsBitMask).eq(0)
    )
    return image.updateMask(mask)

LS8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2") \
        .filterDate('2020-01-01', '2020-12-31') \
        .filterBounds(roi) \
        .map(maskL8sr)

def add_ndvi(image):
    return image.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')

ndvi_image = LS8.map(add_ndvi).median().clip(roi)
Map.addLayer(ndvi_image, {'min': 0, 'max': 1, 'palette': ['white', 'green']}, "NDVI")

# -----------------------------
# DEM
# -----------------------------
dem = ee.Image("USGS/SRTMGL1_003").clip(roi).rename("DEM")
Map.addLayer(dem, {'min': 0, 'max': 2000, 'palette': ['white','brown']}, "DEM")

# -----------------------------
# Climate: WorldClim v2 bioclimatic variables (1km resolution)
# -----------------------------
bio1 = ee.Image("WORLDCLIM/V1/BIO").select('bio01').clip(roi).rename("bio01")  # Mean annual temp
bio12 = ee.Image("WORLDCLIM/V1/BIO").select('bio12').clip(roi).rename("bio12")  # Annual precipitation

# -----------------------------
# Forest canopy ]
# -----------------------------
# # TCC and TCH from Global Forest Canopy Height and Cover
# tcc = ee.Image("ESA/GLOBCOVER_L4_200901_200912_V2_3").clip(roi).rename("TCC")  # approximate
# # GEDI products
tcc = ee.ImageCollection("projects/sat-io/open-datasets/facebook/meta-canopy-height").first().clip(roi).rename("TCC");

# -----------------------------
# Forest canopy / GEDI L2A metrics
# -----------------------------

gedi_l2a_fc = ee.FeatureCollection("LARSE/GEDI/GEDI02_A_002_MONTHLY") \
                .filterBounds(roi) \
                .filter(ee.Filter.date('2019-01-01', '2020-12-31'))


gedi_rh98_img = gedi_l2a_fc.reduceToImage(
    properties=['rh98'],
    reducer=ee.Reducer.mean()
).clip(roi).rename('Rh98')  # no select()


# gedi_rh98_img = gedi_rh98_img.focal_mean(radius=30, units='meters')


# -----------------------------
# Forest canopy / GEDI L2B metrics
# -----------------------------

gedi_l2b_fc = ee.FeatureCollection("LARSE/GEDI/GEDI02_B_002_MONTHLY") \
                .filterBounds(roi) \
                .filter(ee.Filter.date('2019-01-01', '2020-12-31'))


gedi_pai_img = gedi_l2b_fc.reduceToImage(
    properties=['pai'],
    reducer=ee.Reducer.mean()
).clip(roi).rename('PAI')  # no select()


# -----------------------------
# Define square kernel for smoothing
# -----------------------------
kernel = ee.Kernel.square(radius=100)  # 100 m like your JS example

# Apply focal mean directly to single-band images
gedi_pai_img_smooth = gedi_pai_img.focal_mean(kernel=kernel, iterations=1)
gedi_rh98_img_smooth = gedi_rh98_img.focal_mean(kernel=kernel, iterations=1)


# gedi_pai_img = gedi_pai_img.focal_mean(radius=30, units='meters')


# # List of PAVDZ bands
# # pai_bands = [f'pai_z{i}' for i in range(15)]
# pavdz_bands = [f'pavdz_z{i}' for i in range(15)]


# def rasterize_bands(fc, band_list):
#     """Convert a list of point-based bands to a raster image stack."""
#     raster_list = []
#     for band in band_list:
#         img = fc.reduceToImage(properties=[band], reducer=ee.Reducer.mean()) \
#                 .select(['mean']).clip(roi) \
#                 .rename(band) \
#                 .focal_mean(radius=30, units='meters')  # smooth gaps
#         raster_list.append(img)
#     return ee.Image(raster_list)


# # Rasterize PAI and PAVDZ
# # gedi_pai_img = rasterize_bands(gedi_l2b_fc, pai_bands)
# gedi_pavdz_img = rasterize_bands(gedi_l2b_fc, pavdz_bands)

# -----------------------------
# Rasterize GEDI bands with square kernel smoothing
# -----------------------------

def rasterize_bands_smooth(fc, band_list, kernel_radius=100):
    """Convert point-based bands to raster and apply focal mean using a square kernel."""
    raster_list = []

    # Define square kernel (like JS example)
    kernel = ee.Kernel.square(radius=kernel_radius)

    for band in band_list:
        img = fc.reduceToImage(properties=[band], reducer=ee.Reducer.mean()) \
                .select(['mean']).clip(roi) \
                .rename(band)
        # Apply focal mean using the kernel
        img_smooth = img.focal_mean(kernel=kernel, iterations=1)
        raster_list.append(img_smooth)

    # Combine all smoothed rasters into a single image
    return ee.Image(raster_list)


# List of PAVDZ bands
pavdz_bands = [f'pavdz_z{i}' for i in range(15)]

# Rasterize and smooth
gedi_pavdz_img_out = rasterize_bands_smooth(gedi_l2b_fc, pavdz_bands, kernel_radius=100)

# -----------------------------
# Combine all predictors into a single image
# -----------------------------

predictors = ndvi_image.addBands([dem, bio1, bio12, tcc, gedi_pavdz_img_out, gedi_pai_img_smooth, gedi_rh98_img_smooth]) #gedi_pai_img, gedi_rh98_img, gedi_pavdz_img
Map.addLayer(predictors, {}, "All Predictors")

# -----------------------------
# Sample points and train Maxent classifier
# -----------------------------
training = predictors.sampleRegions(
    collection=training_data,
    scale=30
)

classifier = ee.Classifier.amnhMaxent().train(
    features=training,
    classProperty='occurrence',
    inputProperties=predictors.bandNames()
)

# Classify image
image_classified = predictors.classify(classifier)
Map.addLayer(image_classified.select('probability'), {'min':0, 'max':1, 'palette': ['white','light green','green']}, 'Predicted Probability')

Map
